In [1]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=d4b88d9df04dbb8f81dd539c0c09ad3cea0913e8b62559ff96fc907973cc3c02
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [2]:
import wikipedia
import gensim
from gensim import corpora
from gensim.models import CoherenceModel
import pandas as pd
import re
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [8]:
import re
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))
stemmer = WordNetLemmatizer()
Shakti = wikipedia.page('Avengers (comics)')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
def preprocess_text(docu):
    document = re.sub(r'[^a-zA-Z0-9 ]','',str(docu))
    tokens = document.split()
    tokens = [stemmer.lemmatize(word) for word in tokens]
    tokens = [word for word in tokens if word not in en_stop]
    tokens = [word for word in tokens if len(word)>5]

    return tokens
preprocess_data = [];
tokens = preprocess_text(Shakti)
preprocess_data.append(tokens)

In [10]:
input_dict = corpora.Dictionary(preprocess_data)
input_corpora = [input_dict.doc2bow(token, allow_update=True) for token in preprocess_data]

In [16]:
lda_model = gensim.models.ldamodel.LdaModel(input_corpora,num_topics=5,id2word = input_dict,passes=10)
topics = lda_model.print_topics(num_words=2)
print(topics)

[(0, '0.500*"Avengers" + 0.500*"WikipediaPage"'), (1, '0.500*"WikipediaPage" + 0.500*"Avengers"'), (2, '0.500*"Avengers" + 0.500*"WikipediaPage"'), (3, '0.500*"Avengers" + 0.500*"WikipediaPage"'), (4, '0.500*"WikipediaPage" + 0.500*"Avengers"')]


In [13]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=preprocess_data, dictionary=input_dict, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.9999999999999998
